# Japanese Temperature Data Collection
This is a companion to the other Japanese Meteorological Agency notebook on Sakura Data processing.


Sources:
* https://www.data.jma.go.jp/obd/stats/data/mdrr/man/kansoku_gaiyou.html  (smater.index file)
* https://www.data.jma.go.jp/gmd/risk/obsdl/index.php Temperature Request site

Notes:
I'm really hoping this smaster file has the data I need. Well not so much the data as it is the station identifiers that get used in the POST requests to the other link.

Notes2:
This data file has the data I need! HOORAY! Well at least part of it, the station numbers match up with the last 3 digits of the requests I manually put in for Kyoto, Nara, and Osaka

In [16]:
import sys
import pandas as pd
import numpy as np

In [2]:
with open('Japan Temp Data/smaster.index', encoding='shift_jis') as file:
    for i in range(5):
        line = file.readline()
        print(line)
    

401 18343       ﾜﾂｶﾅｲ   WAKKANAI    4524901414070        32    11961010119651231　　　　　　　　　　　　　　　　　　　                            

401 18143       ﾜﾂｶﾅｲ   WAKKANAI    4524901414070        32    11966010119661231　　　　　　　　　　　　　　　　　　　                            

401 18573       ﾜﾂｶﾅｲ   WAKKANAI    4524901414070        32    11967010119671231　　　　　　　　　　　　　　　　　　　                            

401 18578       ﾜﾂｶﾅｲ   WAKKANAI    4524901414070        32    11968010119681231　　　　　　　　　　　　　　　　　　　                            

401 18578       ﾜﾂｶﾅｲ   WAKKANAI    4524901414070       109    11969010119701231　　　　　　　　　　　　　　　　　　　                            



In [3]:
col_widths = [3 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,2 ,1 ,1 ,8 ,12 ,6 ,7,5 ,5 ,3 ,1 ,1 ,8 ,8 ,12 ,18 ,8 ,5 ,12 ,1 ,1 ,1 ,1 ,1 ,1 ,5]

with open('Japan Temp Data/smaster.index', encoding='shift_jis') as file:
    smaster = pd.read_fwf(file,header=None,widths=col_widths,dtype='object')

In [19]:
np.unique(smaster.loc[smaster[14] == 'SAPPORO'][[0,14]])

array(['412', 'SAPPORO'], dtype=object)

In [50]:
site_list = smaster[[0,14]].drop_duplicates()
site_list.rename(columns={0:'Site Id',14:'Site Name'},inplace=True)

site_list['Site Id'] = 's47' + site_list['Site Id']
site_list['Site Name'] = site_list['Site Name'].str.title()

In [52]:
site_list.loc[site_list['Site Name'] == 'Kyoto']

,Site Id,Site Name
1845,s47759,Kyoto


In [56]:
site_list.shape

(174, 2)